In [1]:
import sqlite3
import pandas as pd

In [4]:
conn = sqlite3.connect('FPA_FOD_20170508.sqlite')
df = pd.read_sql_query("SELECT * from Fires", conn)

In [2]:
# load elevation - to collect the weather data
elevation_df = pd.read_csv("elevation.csv")

In [5]:
df.head(2)

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,0511,Plumas National Forest,...,A,40.036944,-121.005833,5.0,USFS,CA,63,063,Plumas,b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0...
1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.933056,-120.404444,5.0,USFS,CA,61,061,Placer,b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\x19^\...


In [6]:
elevation_df.head(2)

,Unnamed: 0,ELEVATION
0,0,924.73
1,1,1831.26


In [8]:
elevation_df.drop("Unnamed: 0", axis=1, inplace=True)
elevation_df.head(1)

,ELEVATION
0,924.73


In [9]:
print(elevation_df.shape, df.shape) # validate both with same number of rows - V

(1880465, 1) (1880465, 39)


# Collect weather

## code to collect

In [12]:
import datetime as dt
import pandas as pd
import numpy as np
from meteostat import Point, Daily

In [13]:
df.head(1)

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,0511,Plumas National Forest,...,A,40.036944,-121.005833,5.0,USFS,CA,63,063,Plumas,b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0...


In [14]:
elevation_df.head(1)

,ELEVATION
0,924.73


**create seprate DISCOVERY date df, and convert it to dates**

In [16]:
date_df = df.DISCOVERY_DATE
date_df.head(1)

0    2453403.5
Name: DISCOVERY_DATE, dtype: float64

In [17]:
date_df = pd.to_datetime(date_df - pd.Timestamp(0).to_julian_date(), unit='D')
date_df.head(1)

0   2005-02-02
Name: DISCOVERY_DATE, dtype: datetime64[ns]

**create df of LATITUDE, LONGITUDE, ELEVATION**

In [19]:
lat_long_el_df = pd.DataFrame({'latitude': np.array(df.LATITUDE),
                                'longitude': np.array(df.LONGITUDE),
                                'elevation': np.array(elevation_df.ELEVATION)})
lat_long_el_df.head(1)

,latitude,longitude,elevation
0,40.036944,-121.005833,924.73


In [25]:
# Number of days before the fire discovery day to collect data start from it.
NUM_DAYS_BEFORE = 1

# Expected num of days in collected data
total_days = NUM_DAYS_BEFORE + 1  

**create the empty new df to fill in the weather data**

In [28]:
def init_Point_object(i):
    # Create Point object for the fire location
    cur_fire_location = lat_long_el_df.iloc[i, :]
    location = Point(cur_fire_location["latitude"],
                     cur_fire_location["longitude"],
                     int(cur_fire_location["elevation"]))

    location.radius = 1e6  # Maximum radius for nearby stations in meters
    location.max_count = 5  # Maximum number of stations used for interpolation
    return location

In [29]:
def update_weather_df(weather_df, i, data):
    # The average air temperature in °C
    weather_df.iloc[i, TAVG_START:TAVG_END_PLUS_ONE] = data['tavg']

    # The minimum air temperature in °C
    weather_df.iloc[i, TMIN_START:TMIN_END_PLUS_ONE] = data['tmin']

    # The maximum air temperature in °C
    weather_df.iloc[i, TMAX_START:TMAX_END_PLUS_ONE] = data['tmax']

    # The average wind speed in km/h
    weather_df.iloc[i, TWSPD_START:TWSPD_END_PLUS_ONE] = data['wspd']

    # The daily precipitation total in mm
    weather_df.iloc[i, PRCP_START:PRCP_END_PLUS_ONE] = data['prcp']

    # The average sea-level air pressure in hPa
    weather_df.iloc[i, PRES_START:PRES_END_PLUS_ONE] = data['pres']

In [27]:
# create the new weather dataframe columns.
cols = ["tavg 0 days before", "tavg 1 days before",
       "tmin 0 days before", "tmin 1 days before",
       "tmax 0 days before", "tmax 1 days before",
       "wspd 0 days before", "wspd 1 days before",
       "prcp 0 days before", "prcp 1 days before",
       "pres 0 days before", "pres 1 days before",]

# columns index range in the new weather df
TAVG_START = 0
TAVG_END_PLUS_ONE = 2
TMIN_START = 2
TMIN_END_PLUS_ONE = 4
TMAX_START = 4
TMAX_END_PLUS_ONE = 6
TWSPD_START = 6
TWSPD_END_PLUS_ONE = 8
PRCP_START = 8
PRCP_END_PLUS_ONE = 10
PRES_START = 10
PRES_END_PLUS_ONE = 12

# new df shape
num_of_weather_data_columns_per_fire = len(cols)
num_of_fires = lat_long_el_df.shape[0]

# creating the weather df
weather_df = pd.DataFrame(np.full((num_of_fires,num_of_weather_data_columns_per_fire), np.nan), columns=cols)

## example - 5 rows

In [36]:
for i in range(5):
    # Set time period
    cur_fire_discovery_day = date_df[i]
    date_NUM_DAYS_BEFORE = cur_fire_discovery_day - dt.timedelta(days=NUM_DAYS_BEFORE)

    # init the Point object fot the fire location
    location = init_Point_object(i)

    # Get daily data for current fire for the fire discovered day and
    # one day before
    data = Daily(location, date_NUM_DAYS_BEFORE,
                 cur_fire_discovery_day)

    data = data.fetch()
    if data.shape[0] == total_days:  # validate that got the all expected days 
                                     # in the collected weather data. if not, skip.
        update_weather_df(weather_df, i, data)

In [37]:
weather_df

,tavg 0 days before,tavg 1 days before,tmin 0 days before,tmin 1 days before,tmax 0 days before,tmax 1 days before,wspd 0 days before,wspd 1 days before,prcp 0 days before,prcp 1 days before,pres 0 days before,pres 1 days before
0,1.1,1.1,-2.8,-2.2,4.5,4.5,4.3,3.2,0.0,0.0,1033.4,1035.9
1,4.0,5.7,-0.5,-2.7,8.4,15.0,10.9,7.3,5.5,0.0,1012.4,1016.3
2,18.7,21.0,14.8,17.0,23.1,24.8,5.7,12.6,0.0,0.0,1016.1,1013.1
3,11.2,11.8,0.4,2.1,19.3,20.4,7.5,6.0,0.0,0.0,1015.5,1015.5
4,11.5,12.1,0.7,2.4,19.6,20.7,7.5,6.0,0.0,0.0,1015.5,1015.5
...,...,...,...,...,...,...,...,...,...,...,...,...
1880460,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1880461,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1880462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1880463,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# all rows
takes a lot of time

In [ ]:
# creating the weather df
weather_df = pd.DataFrame(np.full((num_of_fires,num_of_weather_data_columns_per_fire), np.nan), columns=cols)

for i in range(num_of_fires):
    # Set time period
    cur_fire_discovery_day = date_df[i]
    date_NUM_DAYS_BEFORE = cur_fire_discovery_day - dt.timedelta(days=NUM_DAYS_BEFORE)

    # init the Point object fot the fire location
    location = init_Point_object(i)

    # Get daily data for current fire for the fire discovered day and
    # one day before
    data = Daily(location, date_NUM_DAYS_BEFORE,
                 cur_fire_discovery_day)

    data = data.fetch()
    if data.shape[0] == total_days:  # validate that got the all expected days 
                                     # in the collected weather data. if not, skip.
        update_weather_df(weather_df, i, data)

In [44]:
weather_df

,tavg 0 days before,tavg 1 days before,tmin 0 days before,tmin 1 days before,tmax 0 days before,tmax 1 days before,wspd 0 days before,wspd 1 days before,prcp 0 days before,prcp 1 days before,pres 0 days before,pres 1 days before
0,1.1,1.1,-2.2,-2.8,4.5,4.5,3.2,4.3,0.0,0.0,1035.9,1033.4
1,5.6,3.9,-0.1,-0.6,9.9,8.3,7.3,10.9,0.0,5.5,1016.2,1012.4
2,21.1,18.8,17.1,14.9,24.9,23.2,12.6,5.7,0.0,0.0,1013.1,1016.1
3,11.7,11.1,8.1,7.6,6.4,5.3,6.0,7.5,0.0,0.0,1015.5,1015.5
4,12.0,11.4,8.4,7.9,6.7,5.6,6.0,7.5,0.0,0.0,1015.5,1015.5
...,...,...,...,...,...,...,...,...,...,...,...,...
1880460,24.5,23.7,14.0,12.4,34.6,35.2,4.0,5.8,0.0,0.0,1010.0,1010.5
1880461,20.0,17.6,13.4,10.7,26.8,25.1,7.3,11.2,0.0,0.0,1009.8,1001.9
1880462,24.2,26.1,16.8,16.2,32.9,35.1,9.0,10.4,0.0,0.0,1008.9,1009.1
1880463,25.8,25.8,17.7,17.1,32.1,35.5,11.6,13.9,0.0,0.0,1013.5,1014.4


In [43]:
weather_df.isnull().sum() / ff.shape[0]

tavg 0 days before    0.013292
tavg 1 days before    0.012956
tmin 0 days before    0.004425
tmin 1 days before    0.004445
tmax 0 days before    0.004425
tmax 1 days before    0.004442
wspd 0 days before    0.012668
wspd 1 days before    0.012494
prcp 0 days before    0.037716
prcp 1 days before    0.037207
pres 0 days before    0.057530
pres 1 days before    0.058004
dtype: float64

In [46]:
weather_df.to_csv("weather_df.csv")

In [47]:
med = weather_df.median()
med

tavg 0 days before      17.2
tavg 1 days before      16.9
tmin 0 days before      10.5
tmin 1 days before      10.3
tmax 0 days before      24.1
tmax 1 days before      23.6
wspd 0 days before      10.8
wspd 1 days before      10.4
prcp 0 days before       0.0
prcp 1 days before       0.0
pres 0 days before    1016.0
pres 1 days before    1016.2
dtype: float64

In [50]:
weather_df.fillna(med, inplace=True)

In [51]:
weather_df

,tavg 0 days before,tavg 1 days before,tmin 0 days before,tmin 1 days before,tmax 0 days before,tmax 1 days before,wspd 0 days before,wspd 1 days before,prcp 0 days before,prcp 1 days before,pres 0 days before,pres 1 days before
0,1.1,1.1,-2.2,-2.8,4.5,4.5,3.2,4.3,0.0,0.0,1035.9,1033.4
1,5.6,3.9,-0.1,-0.6,9.9,8.3,7.3,10.9,0.0,5.5,1016.2,1012.4
2,21.1,18.8,17.1,14.9,24.9,23.2,12.6,5.7,0.0,0.0,1013.1,1016.1
3,11.7,11.1,8.1,7.6,6.4,5.3,6.0,7.5,0.0,0.0,1015.5,1015.5
4,12.0,11.4,8.4,7.9,6.7,5.6,6.0,7.5,0.0,0.0,1015.5,1015.5
...,...,...,...,...,...,...,...,...,...,...,...,...
1880460,24.5,23.7,14.0,12.4,34.6,35.2,4.0,5.8,0.0,0.0,1010.0,1010.5
1880461,20.0,17.6,13.4,10.7,26.8,25.1,7.3,11.2,0.0,0.0,1009.8,1001.9
1880462,24.2,26.1,16.8,16.2,32.9,35.1,9.0,10.4,0.0,0.0,1008.9,1009.1
1880463,25.8,25.8,17.7,17.1,32.1,35.5,11.6,13.9,0.0,0.0,1013.5,1014.4


In [ ]:
# can see that in the last row pres 0/1 filled as needed.